In [1]:
import pandas as pd
from pandas import Series, DataFrame

In [23]:
import numpy as np

In [19]:
import patsy

In [54]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
data = DataFrame({ 'x0': [ 1, 2, 3, 4, 5 ], 'x1': [.01, -.01, .25, -4.1, 0 ], 'y': [ -1.5, 0, 3.6, 1.3, -2 ] })

In [3]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [5]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [6]:
df2 = DataFrame(data.values, columns=[ 'one', 'two', 'three' ])

In [7]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [8]:
df3 = data.copy()

In [9]:
df3

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [10]:
df3.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [11]:
model_cols = [ 'x0', 'x1' ]

In [12]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [13]:
data['category'] = pd.Categorical([ 'a', 'b', 'a', 'a', 'b' ], categories=[ 'a', 'b' ])

In [14]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [15]:
dummies = pd.get_dummies(data.category, prefix='category')

In [16]:
data_with_dummies = data.drop('category', axis=1).join(dummies)

In [17]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


In [20]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [21]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [22]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [24]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [25]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [26]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [27]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [28]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [29]:
coef = Series(coef.squeeze(), index=X.design_info.column_names)

In [30]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

In [31]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [32]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [33]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [34]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [35]:
new_data = DataFrame({ 'x0': [ 6, 7, 8, 9, ], 'x1': [ 3.1, -.5, 0, 2.3, ], 'y': [ 1, 2, 3, 4, ] })

In [36]:
new_X = patsy.build_design_matrices([ X.design_info ], new_data)

In [37]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [38]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [39]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

In [41]:
data = DataFrame({ 'key1': [ 'a', 'a', 'b', 'b', 'a', 'b', 'a', 'b' ], 'key2': [ 0, 1, 0, 1, 0, 1, 0, 0, ], 'v1': [ 1, 2, 3, 4, 5, 6, 7, 8, ], 'v2': [ -1, 0, 2.5, -.5, 4, -1.2, .2, -1.7 ] })

In [42]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [43]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [44]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [45]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [46]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [47]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [48]:
data['key2'] = data['key2'].map({ 0: 'zero', 1: 'one' })

In [49]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [50]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [51]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [52]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [53]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

In [55]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + variance * np.random.randn(*size)

In [56]:
np.random.seed(12345)

In [57]:
N = 100

In [58]:
X = np.c_[ dnorm(0, .4, N), dnorm(0, .6, N), dnorm(0, .2, N) ]

In [59]:
eps = dnorm(0, .1, N)

In [60]:
beta = [ .1, .3, .5 ]

In [61]:
y = np.dot(X, beta) + eps

In [62]:
X[:5]

array([[-0.08188306, -0.93939438,  0.22549622],
       [ 0.19157734, -0.33752411, -0.11367269],
       [-0.20777549, -0.01959848,  0.06187243],
       [-0.22229212, -0.55740372, -0.11547709],
       [ 0.78631223, -0.28954359, -0.23372681]])

In [63]:
y[:5]

array([-0.00253515, -0.27996046, -0.03354503, -0.28177094, -0.08708901])

In [64]:
X_model = sm.add_constant(X)

In [65]:
X_model[:5]

array([[ 1.        , -0.08188306, -0.93939438,  0.22549622],
       [ 1.        ,  0.19157734, -0.33752411, -0.11367269],
       [ 1.        , -0.20777549, -0.01959848,  0.06187243],
       [ 1.        , -0.22229212, -0.55740372, -0.11547709],
       [ 1.        ,  0.78631223, -0.28954359, -0.23372681]])

In [66]:
model = sm.OLS(y, X)

In [67]:
result = model.fit()

In [68]:
result.params

array([0.13913054, 0.26858106, 0.50067241])

In [69]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     100.3
Date:                Mon, 18 Mar 2019   Prob (F-statistic):           1.28e-29
Time:                        21:01:09   Log-Likelihood:                 80.825
No. Observations:                 100   AIC:                            -155.6
Df Residuals:                      97   BIC:                            -147.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1391      0.026      5.251      0.0

In [70]:
data = DataFrame(X, columns=[ 'col0', 'col1', 'col2' ])

In [71]:
data['y'] = y

In [72]:
data[:5]

,col0,col1,col2,y
0,-0.081883,-0.939394,0.225496,-0.002535
1,0.191577,-0.337524,-0.113673,-0.279960
2,-0.207775,-0.019598,0.061872,-0.033545
3,-0.222292,-0.557404,-0.115477,-0.281771
4,0.786312,-0.289544,-0.233727,-0.087089


In [73]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

In [74]:
results.params

Intercept    0.010612
col0         0.138074
col1         0.269310
col2         0.510471
dtype: float64

In [75]:
results.tvalues

Intercept     0.952188
col0          5.204386
col1         14.232764
col2          8.840053
dtype: float64

In [76]:
results.predict(data[:5])

0   -0.138573
1   -0.111861
2    0.008230
3   -0.229143
4   -0.078106
dtype: float64

In [77]:
init_x = 4

In [78]:
values = [ init_x ] * 2

In [79]:
N = 1000

In [80]:
b0 = .8

In [81]:
b1 = -.4

In [82]:
noise = dnorm(0, .1, N)

In [83]:
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [84]:
MAXLAGS = 5

In [85]:
model = sm.tsa.AR(values)

In [86]:
results = model.fit(MAXLAGS)

In [87]:
results.params

array([-0.00208441,  0.78358666, -0.40862768, -0.01428997,  0.00320974,
        0.01504663])